# Basic Chat with Storm API

Learn how to chat with your documents using Storm's RAG capabilities.

## Setup

In [2]:
!pip install requests

import requests
import json
import os

# Configuration
API_KEY = os.getenv("STORM_API_KEY", "st_fd22b63dc2304bcc9da18744ca462631")
API_URL = "https://live-stargate.sionic.im"

headers = {"storm-api-key": API_KEY}

print("✅ Setup complete")

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]
✅ Setup complete


## 1. Simple Chat Request

Send a question and get an AI-powered answer based on your documents.

In [5]:
def chat_with_documents(query, variables=None):
    """Send a chat request to Storm API."""
    
    data = {"question": query}
    if variables:
        data["variables"] = variables
    
    response = requests.post(
        f"{API_URL}/api/v2/answer",
        headers=headers,
        json=data
    )
    
    if response.status_code == 200:
        result = response.json()["data"]
        return result
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
        return None

# Test with a simple question
question = "can you promote Sea World Bangkok Ocean world the Riverfront to young students?"
print(f"🤔 Question: {question}\n")

result = chat_with_documents(question)

if result:
    chat = result["chat"]
    contexts = result["contexts"]
    
    print(f"💬 Answer: {chat['answer']}\n")
    print(f"📚 Used {len(contexts)} sources")
    print(f"🧵 Thread ID: {chat['threadId']}")
    print(f"✅ Status: {chat['status']}")

🤔 Question: can you promote Sea World Bangkok Ocean world the Riverfront to young students?

💬 Answer: "🌊🐠 มาเปิดโลกใต้ทะเลที่ Sea Life Bangkok Ocean World กันเถอะ! ที่นี่มีทั้งแนวปะการังสีสันสดใส อุโมงค์ใต้ทะเลที่ให้ความรู้สึกเหมือนเดินอยู่ใต้น้ำ และชาร์ค วอล์คที่ตื่นเต้นสุดๆ! เหมาะสำหรับน้องๆ ที่อยากเรียนรู้และสนุกไปพร้อมกัน! เปิดทุกวัน 10.00 - 20.00 น. #SeaLifeAdventure #เรียนรู้สนุกสนาน"

📚 Used 91 sources
🧵 Thread ID: 7361670852860026880
✅ Status: success


## 2. Understanding Context Sources

Storm API provides context sources for each answer - the documents and snippets used.

In [ ]:
def display_contexts(contexts):
    """Display context sources in a readable format."""
    
    if not contexts:
        print("No contexts provided")
        return
    
    print(f"📚 Context Sources ({len(contexts)} total):\n")
    
    for i, ctx in enumerate(contexts, 1):
        print(f"Source {i}:")
        print(f"  📦 Bucket: {ctx['bucketName']}")
        print(f"  📄 File: {ctx['fileName']}")
        print(f"  📑 Page: {ctx['pageName']}")
        print(f"  🔢 Reference: [{ctx['referenceIdx']}]")
        print(f"  📝 Type: {ctx['type']}")
        print(f"\n  Context snippet:")
        print(f"  \"{ctx['context'][:200]}...\"" if len(ctx['context']) > 200 else f"  \"{ctx['context']}\"")
        print()

# Show contexts from previous answer
if 'result' in locals() and result:
    display_contexts(result['contexts'])

## 3. Conversation Threading

Maintain conversation context across multiple questions.

In [7]:
class ChatSession:
    """Manage a chat session with Storm API."""
    
    def __init__(self, api_key, api_url="https://live-stargate.sionic.im"):
        self.headers = {"storm-api-key": api_key}
        self.api_url = api_url
        self.thread_id = None
        self.history = []
    
    def ask(self, question):
        """Ask a question in the current thread."""
        
        response = requests.post(
            f"{self.api_url}/api/v2/answer",
            headers=self.headers,
            json={"question": question}
        )
        
        if response.status_code == 200:
            data = response.json()["data"]
            chat = data["chat"]
            
            # Store thread ID from first response
            if not self.thread_id:
                self.thread_id = chat["threadId"]
            
            # Add to history
            self.history.append({
                "question": question,
                "answer": chat["answer"],
                "contexts": len(data["contexts"])
            })
            
            return chat["answer"], data["contexts"]
        
        return None, None
    
    def show_history(self):
        """Display conversation history."""
        print(f"🧵 Thread: {self.thread_id}\n")
        
        for i, turn in enumerate(self.history, 1):
            print(f"Turn {i}:")
            print(f"  Q: {turn['question']}")
            print(f"  A: {turn['answer'][:100]}..." if len(turn['answer']) > 100 else f"  A: {turn['answer']}")
            print(f"  📚 {turn['contexts']} sources used")
            print()

# Create a chat session
session = ChatSession(API_KEY)

# Have a conversation
questions = [
    "What is RAG?",
    "How does it work?",
    "What are the benefits?"
]

print("💬 Starting conversation...\n")

for q in questions:
    print(f"Q: {q}")
    answer, contexts = session.ask(q)
    if answer:
        print(f"A: {answer[:150]}..." if len(answer) > 150 else f"A: {answer}")
        print(f"📚 Sources: {len(contexts)}\n")
    else:
        print("❌ Failed to get answer\n")

💬 Starting conversation...

Q: What is RAG?
A: RAG หรือ Retrieval-Augmented Generation เป็นเทคนิคที่ใช้ในการประมวลผลภาษาธรรมชาติ โดยการรวมการดึงข้อมูล (Retrieval) และการสร้างข้อความ (Generation) เข...
📚 Sources: 79

Q: How does it work?
A: การทำงานของ TikTok Commerce Assistant Bot คือการช่วยให้คุณเข้าใจและใช้ประโยชน์จากเทรนด์การตลาดที่กำลังมาแรงบน TikTok เพื่อเพิ่มยอดขายและการมีส่วนร่วมข...
📚 Sources: 84

Q: What are the benefits?
A: ประโยชน์ของการเที่ยวกรุงเทพฯ มีเพียบเลย! 🌟 ไม่ว่าจะเป็นการสัมผัสวัฒนธรรมไทยที่หลากหลาย ชิมอาหารสตรีทฟู้ดสุดฟิน หรือช้อปปิ้งในตลาดนัดที่มีของให้เลือกมา...
📚 Sources: 88



## 4. Advanced Chat Features

In [ ]:
def chat_with_context(query, bucket_ids=None, thread_id=None):
    """Chat with specific buckets and thread context."""
    
    # Note: The /api/v2/answer endpoint doesn't support bucket filtering
    # This is shown for the streaming endpoint structure
    
    print("📝 Advanced Chat Parameters:\n")
    
    print("1. Thread Management:")
    print("   • New conversation: Don't specify thread_id")
    print("   • Continue conversation: Use existing thread_id")
    print("   • Each thread maintains conversation history")
    
    print("\n2. Variables (for template queries):")
    print("   • Pass dynamic values to queries")
    print("   • Example: {'product': 'Storm API', 'version': '2.0'}")
    
    print("\n3. Answer Structure:")
    print("   • chat.answer: The AI response")
    print("   • chat.status: Processing status")
    print("   • contexts: Source documents used")
    
    # Example with variables
    query_template = "Tell me about {topic} in {domain}"
    variables = {
        "topic": "document processing",
        "domain": "Storm API"
    }
    
    print("\n📋 Example with variables:")
    print(f"Query: {query_template}")
    print(f"Variables: {variables}")
    
    # Note: Variables would be processed by Storm API
    # to create the final query

## 5. Error Handling and Edge Cases

In [ ]:
def robust_chat(query, max_retries=3):
    """Chat with error handling and retries."""
    
    for attempt in range(max_retries):
        try:
            response = requests.post(
                f"{API_URL}/api/v2/answer",
                headers=headers,
                json={"query": query},
                timeout=30  # 30 second timeout
            )
            
            if response.status_code == 200:
                data = response.json()
                if data["status"] == "success":
                    return data["data"]
                else:
                    print(f"API Error: {data}")
            
            elif response.status_code == 429:
                # Rate limited
                wait_time = int(response.headers.get("Retry-After", 60))
                print(f"Rate limited. Waiting {wait_time}s...")
                time.sleep(wait_time)
                continue
            
            elif response.status_code == 401:
                print("❌ Authentication failed. Check API key.")
                return None
            
            else:
                print(f"HTTP Error {response.status_code}: {response.text}")
        
        except requests.exceptions.Timeout:
            print(f"Timeout on attempt {attempt + 1}")
        
        except requests.exceptions.ConnectionError:
            print(f"Connection error on attempt {attempt + 1}")
        
        except Exception as e:
            print(f"Unexpected error: {e}")
        
        if attempt < max_retries - 1:
            print("Retrying...")
            time.sleep(2 ** attempt)
    
    return None

# Test edge cases
test_queries = [
    "Normal question about Storm API",
    "",  # Empty query
    "A" * 5000,  # Very long query
    "What is 什么是 🤔?",  # Unicode and emojis
]

print("🧪 Testing edge cases:\n")

for i, query in enumerate(test_queries, 1):
    print(f"Test {i}: {query[:50]}..." if len(query) > 50 else f"Test {i}: '{query}'")
    
    if not query:
        print("  ⚠️  Skipping empty query\n")
        continue
    
    result = robust_chat(query)
    if result:
        print("  ✅ Success\n")
    else:
        print("  ❌ Failed\n")

## 6. Building a Simple Chatbot

In [ ]:
class StormChatbot:
    """Interactive chatbot using Storm API."""
    
    def __init__(self, api_key, api_url="https://https://live-stargate.sionic.im"):
        self.api_key = api_key
        self.api_url = api_url
        self.headers = {"storm-api-key": api_key}
        self.thread_id = None
    
    def chat(self, message):
        """Send message and get response."""
        
        response = requests.post(
            f"{self.api_url}/api/v2/answer",
            headers=self.headers,
            json={"query": message}
        )
        
        if response.status_code == 200:
            data = response.json()["data"]
            chat = data["chat"]
            contexts = data["contexts"]
            
            # Store thread ID
            if not self.thread_id:
                self.thread_id = chat["threadId"]
            
            return {
                "answer": chat["answer"],
                "sources": len(contexts),
                "status": "success"
            }
        else:
            return {
                "answer": "Sorry, I couldn't process your request.",
                "sources": 0,
                "status": "error"
            }
    
    def format_response(self, response):
        """Format response for display."""
        answer = response["answer"]
        sources = response["sources"]
        
        # Add source indicator
        if sources > 0:
            footer = f"\n\n📚 Based on {sources} sources"
        else:
            footer = "\n\n💭 General response"
        
        return answer + footer

# Create chatbot
bot = StormChatbot(API_KEY)

# Simulate conversation
print("🤖 Storm Chatbot Demo\n")
print("Type 'quit' to exit\n")

# Demo conversation
demo_messages = [
    "Hello! What can you help me with?",
    "Tell me about document processing",
    "How accurate is the RAG system?"
]

for msg in demo_messages:
    print(f"You: {msg}")
    response = bot.chat(msg)
    print(f"Bot: {bot.format_response(response)}")
    print()

# Interactive mode (commented out for notebook)
# while True:
#     user_input = input("You: ")
#     if user_input.lower() == 'quit':
#         break
#     
#     response = bot.chat(user_input)
#     print(f"Bot: {bot.format_response(response)}\n")

## 7. Best Practices

In [ ]:
print("📚 Chat API Best Practices:\n")

print("1. Query Design:")
print("   • Be specific and clear")
print("   • Provide context when needed")
print("   • Avoid overly broad questions")
print("   • Use natural language")

print("\n2. Thread Management:")
print("   • Use threads for related questions")
print("   • Start new threads for new topics")
print("   • Don't mix unrelated conversations")

print("\n3. Performance:")
print("   • Cache frequently asked questions")
print("   • Implement client-side rate limiting")
print("   • Use appropriate timeouts")

print("\n4. Error Handling:")
print("   • Always check response status")
print("   • Provide fallback responses")
print("   • Log errors for debugging")

# Example: Well-structured query patterns
good_queries = [
    "What are the key features of Storm API?",
    "How do I upload documents using the Python SDK?",
    "Explain the difference between DEFAULT and STORM_PARSE parsers",
    "What are the rate limits for the chat API?"
]

poor_queries = [
    "Tell me everything",
    "API?",
    "How?",
    "Explain all features and capabilities and use cases"
]

print("\n✅ Good Query Examples:")
for q in good_queries:
    print(f"   • {q}")

print("\n❌ Poor Query Examples:")
for q in poor_queries:
    print(f"   • {q}")

## Summary

You've learned how to:
- ✅ Send chat requests to Storm API
- ✅ Understand context sources
- ✅ Manage conversation threads
- ✅ Handle errors properly
- ✅ Build a simple chatbot
- ✅ Follow best practices

## Next Steps

- [Streaming Chat](./02-streaming-chat.ipynb) - Real-time streaming responses
- [Context Search](./03-context-search.ipynb) - Search for relevant contexts
- [Building a Chatbot](./04-chatbot-example.ipynb) - Complete chatbot example